In [1]:
# https://www.nih.gov/news-events/news-releases/nih-clinical-center-provides-one-largest-publicly-available-chest-x-ray-datasets-scientific-community

import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import os
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms as tf
from os import walk
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from matplotlib import image
from torchvision import transforms
# Set ipython's max row display
pd.set_option('display.max_row', 1000)

# Set iPython's max column width to 50
pd.set_option('display.max_columns', 50)

In [2]:
train_df = pd.read_csv('train_df2.csv')
test_df = pd.read_csv('test_df2.csv')

In [3]:
train_df.sample(10)

,Finding Label,Image Index,h,w,x,y
12372,1,00003867_016.png,2991.0,2810.0,0.143,0.143
59846,1,00018544_001.png,2048.0,2500.0,0.168,0.168
76675,8,00025556_021.png,2991.0,2674.0,0.143,0.143
39457,3,00012161_003.png,2991.0,2992.0,0.143,0.143
74486,1,00024296_000.png,2500.0,2048.0,0.168,0.168
6383,1,00002060_001.png,2544.0,3056.0,0.139,0.139
28266,1,00008808_002.png,2048.0,2500.0,0.171,0.171
67083,1,00020939_000.png,2991.0,2838.0,0.143,0.143
1286,4,00000459_048.png,2540.0,3056.0,0.139,0.139
38728,8,00011960_009.png,2048.0,2500.0,0.168,0.168


In [4]:
# (train_df['Image Index'] == '00030181_001.png').value_counts()
# 00028247_001.png

In [5]:
import os
filename_list = []
for root, dirs, files in os.walk("datasets/data/images", topdown=True):
    for name in files:
        filename_list.append(name)
#         with Image.open(os.path.join("datasets/data/images", name)) as f:
#             print(len(f.size))

In [6]:
train_df['Image Index'].isin(filename_list).value_counts()

True     82995
False     3529
Name: Image Index, dtype: int64

In [7]:
train_df = train_df[train_df['Image Index'].isin(filename_list)]

In [8]:
train_df['Image Index'].isin(filename_list).value_counts()

True    82995
Name: Image Index, dtype: int64

In [9]:
train_df['Image Index'].isna().value_counts()

False    82995
Name: Image Index, dtype: int64

In [10]:
test_df = test_df[test_df['Image Index'].isin(filename_list)]

In [11]:
test_df.sample(10)

,Finding Label,Image Index,h,w,x,y
12921,4,00015792_001.png,2991.0,2992.0,0.143,0.143
17907,4,00020945_016.png,2544.0,3000.0,0.139,0.139
8540,1,00012141_025.png,2048.0,2500.0,0.168,0.168
9510,1,00013249_053.png,2048.0,2500.0,0.168,0.168
10488,4,00013922_018.png,2544.0,3056.0,0.139,0.139
16534,0,00019363_042.png,2544.0,3056.0,0.139,0.139
12559,7,00015530_083.png,2544.0,3056.0,0.139,0.139
17113,8,00020146_011.png,2991.0,2666.0,0.143,0.143
4571,1,00007444_001.png,2048.0,2500.0,0.171,0.171
7506,8,00011366_005.png,2048.0,2500.0,0.168,0.168


In [12]:
class RescalePIL(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image, label = sample['image'], sample['label']

        h, w = image.size[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)

        img = np.array(image.resize((new_h, new_w)))

        return {'image': img, 'label': label}

In [13]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, label = sample['image'], sample['label']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
#         image = image.transpose((0, 1))
#         print(image.shape)
        return {'image': torch.from_numpy(image).unsqueeze(0),
                'label': torch.from_numpy(label)}

In [14]:

class MNISTDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, dataframe, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.dataframe = dataframe
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.dataframe['Image Index'][idx])
        my_image = np.array(Image.open(img_name))
        if len(my_image.shape) > 2 and my_image.shape[2] == 4:
            my_image = my_image[:,:,1]
        row = self.dataframe.iloc[idx]
        label = np.array([row['Finding Label']])
        sample = {'image': my_image, 'label': label}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [15]:
my_train_set = MNISTDataset(train_df, 'datasets/data/images', transform=transforms.Compose([
#         RescalePIL(200),
        ToTensor()
]))

In [ ]:
# my_image = my_train_set.__getitem__(4)['image']
# my_image.shape
# #imshow will work if it's h,w,c
# #torch is c h w
# #not sure about h and w
# plt.imshow(my_image)

In [ ]:
# plt.imshow(my_image.squeeze(0));

In [16]:
batch_loader_params = {
    "batch_size": 2,
    "shuffle": True,
    "num_workers": 1
}
dataloader = DataLoader(my_train_set, **batch_loader_params)

In [ ]:
# iter(dataloader).next()
# for i, each in enumerate(dataloader):
#     print(i, each['image'].shape)

In [ ]:
# batch_samples = iter(dataloader)
# samples = batch_samples.next()
# datset_batch = torchvision.utils.make_grid(samples['image'])

In [ ]:
# plt.figure(figsize=(20,10))
# for index, each in enumerate(datset_batch):
#     plt.imshow(each.squeeze(0))

In [17]:
# 1x200x200

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)  # 28x28x4 with batch of 6, 18816
#         self.conv2 = nn.Conv2d(20, 50, 5, 1)
#         self.fc1 = nn.Linear(4*4*50, 60)
#         self.fc2 = nn.Linear(60, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))  # 24x24x20 with at batch of 6
        return x

In [18]:
net = Net()

In [19]:
def spatial_size(input_size: int, kernel_size: int, stride: int = 1, padding: int = 0):
    # https://cs231n.github.io/convolutional-networks/
    spatial_size = (input_size - kernel_size + 2 * padding)/stride + 1
    assert spatial_size % 1 == 0
    assert spatial_size > 0
    return int(spatial_size)

In [20]:
print(spatial_size(1024, 500))

525


In [21]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [22]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 100, 525, 1)  
        self.conv2 = nn.Conv2d(100, 50, 50, 1)
        self.fc1 = nn.Linear(4*4*50, 60)
        self.fc2 = nn.Linear(60, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
#         x = F.max_pool2d(x, 2, 2) 
#         x = F.relu(self.conv2(x)) 
#         x = F.max_pool2d(x, 2, 2) 
#         x = x.view(-1, 4*4*50)    
#         x = F.relu(self.fc1(x))   
#         x = self.fc2(x)           
        # # There's no activation at the final layer because of the criterion of CEL
        return x


net = Net()

In [23]:
for epoch in range(1):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(dataloader):
        # get the inputs; data is a list of [inputs, labels]
#         inputs, labels = data['image'], data['label']
        print(data['image'].shape)
        x, y = data['label'], data['image']

#         # zero the parameter gradients
#         optimizer.zero_grad()
#         # forward + backward + optimize
#         outputs = net(data['image'])
#         loss = criterion(outputs, data['label'].squeeze())
#         loss.backward()
#         optimizer.step()

#         # print statistics
#         running_loss += loss.item()

#     print('[%d, %5d] loss: %.5f' %
#           (epoch + 1, i + 1, running_loss / (epoch*i+1)))

torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Si

torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Si

torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Si

torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Size([2, 1, 1024, 1024])
torch.Si

KeyboardInterrupt: 